# Bim File

variant identifier and minor allele

In [1]:
import pandas as pd
df_bim=pd.read_csv('Data/0_Raw/DataS1.bim', sep='\t', header=None)
# df_bim = 1 and 4 columns
df_bim = df_bim[[1, 4]]

df_bim.rename(columns={1: 'VariantID', 4: 'minor'}, inplace=True)

In [2]:
# conver df_bim['variantID'] to a list
columns= df_bim['VariantID'].to_list()
columns.insert(0, 'familyID')
columns.insert(1, 'individualID')
columns.insert(2, 'fatherID')
columns.insert(3, 'motherID')
columns.insert(4,'Sex')
columns.insert(5, 'date')

In [3]:
minor=df_bim['minor'].to_list()
minor.insert(0, 'None')
minor.insert(1, 'None')
minor.insert(2, 'None')
minor.insert(3, 'None')
minor.insert(4,'None')
minor.insert(5, 'None')

# Ped File

familly id and individual id and allels

## Split the ped file by familly id

In [4]:
! rm -r Data/1_raw2freq/countries/*

In [5]:
import os
 
def split_and_convert(file_path):
    # Create a directory to store output files
    os.makedirs('Data/1_raw2freq/countries', exist_ok=True)
    populations = []
    with open(file_path, 'r') as ped_file:
        for row in ped_file:
            row_data = row.strip().split()
            populations.append(row_data[0])
            pedigree = row_data[0]
            with open(f'Data/1_raw2freq/countries/{pedigree}.txt', 'a') as out_file:
                # Join the pairs of alleles in each row
                new_fields = row_data[:6]
                for i in range(6, len(row_data), 2):
                    new_fields.append(row_data[i] + row_data[i+1])
                # Write the new fields to the output file
                out_file.write(' '.join(new_fields) + '\n')
        
    return list(set(populations))

file_path = 'Data/0_Raw/DataS1.ped'

populations = split_and_convert(file_path)
print(populations)



['Norway', 'Croatia', 'CzechRepublic', 'Germany', 'Romania', 'Sweden', 'Turkey', 'Armenia', 'Kazakhstan', 'Switzerland', 'Estonia', 'TheNetherlands', 'Denmark', 'Greece', 'Poland', 'Jordan', 'Spain', 'Luxembourg', 'Lithuania', 'UnitedKingdom', 'Austria', 'Georgia', 'Macedonia', 'Bulgaria', 'Russia', 'Israel', 'Latvia', 'Iran', 'Italy', 'Portugal', 'France', 'Serbia', 'Ukraine', 'Hungary']


In [6]:
dfeuro = pd.read_excel(
    'Data/0_Raw/Eurasian - Dataset_tims.xlsx', sheet_name='Eurasian')
dfeuro = pd.DataFrame(dfeuro)
dfeuro.rename(columns={
              'Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]': 'date'}, inplace=True)

# date_range is a list of the range of dates from 0 to 12000 years ago in 1000 year intervals
date_range = list(range(0, 12000, 1000)) 
# if dfeuro['date'] is in the range of date_range, then dfeuro['cat_date'] is the index of the range
dfeuro['cat_date']=pd.cut(dfeuro['date'], date_range, labels=range(0, len(date_range)-1))


In [7]:

def freq(pop_path, country):
    '''This function takes a population file and returns a dictionary of the frequency of each variant'''
    df = pd.read_csv(pop_path, sep=' ', header=None)
    df.columns = columns
    df.date = 0
    # print(len(df))
    # locate df['individualID'] in dfeuro['Master ID'] and add df['cat_date'] to the df[date]
    for i in range(0, len(df)):
        for j in range(0, len(dfeuro)):
            if df['individualID'][i] == dfeuro['Master ID'][j]:
                df['date'][i] = dfeuro['cat_date'][j]
  
    # get unique dates in df['date'] and store in a list and remove None
    date = df['date'].unique()
    date = date.tolist()
    if 0 in date:
        date.remove(0)
    

    countryfreq = {}
    
    for d in date:
        freqdate= []
        df2 = df[df['date'] == d]
        for i in range(6, len(df.columns)):
            minorAllele = minor[i]
            count = 0
            total = len(df2)*2
            for j in range(0, len(df2)):
                # count occurrences of the minor allele in column i
                count += str(df2.iloc[j, i]).count(str(minorAllele))
            freqdate.append(count/total)
            
        name= f'{country}_{d}'
        countryfreq[name] = freqdate
        
    return countryfreq





In [8]:
import multiprocessing
! rm Data/S1_freq.csv
def process_country(country):
    pop_path = f'Data/1_raw2freq/countries/{country}.txt'
    freq_results = freq(pop_path, country)
    with open('Data/1_raw2freq/S1_freq.csv', 'a') as f:
        for key, value in freq_results.items():
            str_values = [str(val) for val in value]  # convert float values to strings
            f.write('%s\t%s\n' % (key, '\t'.join(str_values)))

if __name__ == '__main__':
    with multiprocessing.Pool() as pool:
        pool.map(process_country, populations)

## Filter SNPs which have 0 in more than 80% of the countries-age groups

In [ ]:
import pandas as pd

# open S1_freq.csv.gz and make a df and calculate the mean of each column
df = pd.read_csv('Data/1_raw2freq/S1_freq.csv', sep='\t', header=None)

In [16]:
dff=[]
for i in range(0, len(df.columns)):
   # count the number of times a value == 0 in each column
    count = df[i].value_counts().get(0)
    dff.append(count)

In [17]:
df2=df.copy()
# get indeces of dff where that are > 80
index = [i for i, x in enumerate(dff) if x > 95]
# remove df2 columns that are in index
df2.drop(df2.columns[index], axis=1, inplace=True)
df2.shape

(119, 1058)

In [18]:
# save to csv without index and header
# df2.to_csv('Data/1_raw2freq/S1_freq_filtered.csv', index=False, header=False, sep='\t')

df2.to_csv('Data/1_raw2freq/test.csv', index=False, header=False, sep='\t')


In [19]:
# convert every *.csv file in the directory to *.csv.gz
! gzip -9 Data/1_raw2freq/*.csv


In [20]:
import pandas as pd

df=pd.read_csv('Data/1_raw2freq/test.csv.gz', sep='\t', header=None, compression='gzip')

# convert first column to a list
pop = df[0].tolist()
cont=[]
date=[]
for i in range(0, len(pop)):
    # split the first column at '_' and store the first element in a list
    x = pop[i].split('_')
    cont.append(x[0])
    date.append((int(x[1]))*1000)

# add cont and date to df as columns 1 and 2
df.insert(1, 'country', cont)
df.insert(2, 'date', date)

# drop the first column
df.drop(df.columns[0], axis=1, inplace=True)

columns= ['country']+['date'] + [f'SNP{str(i)}' for i in range(1, len(df.columns)-1)]
df.columns = columns
df
df.to_csv('Data/1_raw2freq/test_1.csv', sep='\t')
    

In [21]:
df

,country,date,SNP1,SNP2,SNP3,SNP4,SNP5,SNP6,SNP7,SNP8,...,SNP1048,SNP1049,SNP1050,SNP1051,SNP1052,SNP1053,SNP1054,SNP1055,SNP1056,SNP1057
0,Denmark,3000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Denmark,4000,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,...,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000
2,Denmark,2000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
3,Estonia,6000,0.500000,1.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.000000,0.500000,0.000000,0.500000,0.500000,0.000000,0.500000,1.000000,0.500000,0.000000
4,Estonia,5000,1.000000,0.000000,1.000000,0.000000,0.000000,0.500000,0.000000,0.500000,...,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Hungary,2000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
115,UnitedKingdom,3000,0.326087,0.565217,0.500000,0.434783,0.347826,0.434783,0.413043,0.282609,...,0.434783,0.369565,0.304348,0.347826,0.543478,0.413043,0.326087,0.391304,0.456522,0.500000
116,UnitedKingdom,5000,0.193548,0.290323,0.419355,0.419355,0.258065,0.451613,0.419355,0.580645,...,0.193548,0.290323,0.451613,0.483871,0.483871,0.387097,0.483871,0.548387,0.354839,0.419355
117,UnitedKingdom,4000,0.333333,0.424242,0.393939,0.333333,0.242424,0.333333,0.272727,0.303030,...,0.272727,0.424242,0.393939,0.333333,0.454545,0.363636,0.484848,0.303030,0.363636,0.363636
